# Project 8 Chase and Joseph

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import random
from sklearn.tree import export_graphviz
from IPython.display import SVG
from graphviz import Source
from sklearn.metrics import precision_recall_fscore_support
import random
from itertools import combinations
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import seaborn as sns

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

import matplotlib.pyplot as plt

### Read in data

In [2]:
data = pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Describe data

In [ ]:
for col in data.columns:
    display(data[col].describe())

### Plot each column vs outcome

In [ ]:
for col in data.columns:
    plt.figure()
    plt.title('Diabetes vs '+col)
    ax = sns.regplot(x='Outcome', y=col, data=data)
    print(col + " done")

### Create feature columns

In [ ]:
feature = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
y = data['Outcome']
x = data[feature]

### Run every combination of columns through tree classifier

In [ ]:
outputScores = []
outputUsage = []
new_column_names = ['cols', 'Precision-0', 'Precision-1', 'Recall-0', 'Recall-1', 'F0-Score', 'F1-Score']
for item in feature:
    outputUsage.append({'score': 0, 'times': 0, 'name': item})

for i in range(1, len(feature) + 1):
    tuples = combinations(range(len(feature)), i)
    for tup in tuples:
        tupList = list(tup)
        # print(tupList)
        theX = x.iloc[: , tupList]
        treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
        treeclf.fit(theX, y)

        y_pred = treeclf.predict(theX)

        for j in range(len(tupList)):
            outputUsage[tupList[j]]['score'] += treeclf.feature_importances_[j]
            outputUsage[tupList[j]]['times'] += 1
         
        p,r,f,s = precision_recall_fscore_support(y, y_pred)
        colNames = [feature[i] for i in tupList]
        outputScores.append([', '.join(colNames), p[0], p[1], r[0], r[1], f[0], f[1]])

### Get average of score

In [ ]:
for item in outputUsage:
    item['score'] = item['score'] / item['times']

### Create dataframe

In [ ]:
frame = pd.DataFrame(data = outputScores, columns = new_column_names)
frame.head(8)

### Show top F-Scores

In [ ]:
cols = frame['cols']
main = frame.drop(columns=['cols'])

index0 = main['F0-Score'].idxmax()
index1 = main['F1-Score'].idxmax()

f0 = main.loc[index0]
f1 = main.loc[index1]
print(cols.loc[index0])
print(f0)
print(cols.loc[index1])
print(f1)

### Plot feature importances

In [ ]:
usageFrame = pd.DataFrame.from_dict(outputUsage)
usage = usageFrame.drop(columns=["times"])
fig_dims = (10, 8)

fig, ax = plt.subplots(figsize=fig_dims)
g = sns.barplot(data=usage, x="name", y="score", ax=ax)
g.set_xticklabels(g.get_xticklabels(), rotation=30, horizontalalignment='right')

### Tree classfier top 5 features (depth 3)

In [ ]:
feature = ['Pregnancies', 'Glucose', 'Insulin', 'Age', 'BMI']
x = data[feature]
treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(x, y)

graph = Source(tree.export_graphviz(treeclf, out_file=None,
                                    feature_names=feature,
#                                     class_names=['0', '1'], filled = True))
                                    class_names=['0', '1'], filled = True))
svg = SVG(graph.pipe(format='svg'))
display(svg)

graph.format = 'png'
graph.view(filename='digraph3', directory='./')

# Get a random sample to see how it does
# i = random.randint(0, n)
i = 555 # an interesting case
# i = 15 # an interesting case
sample = data.iloc[i]
print('Sample patient {} to test on. Tree classified as {}'.format(i, treeclf.predict([sample[feature]])))
display(sample)

### Tree classfier top 5 features (depth 5)

In [ ]:
feature = ['Pregnancies', 'Glucose', 'Insulin', 'Age', 'BMI']
x = data[feature]
treeclf = DecisionTreeClassifier(max_depth=5, random_state=1)
treeclf.fit(x, y)

graph = Source(tree.export_graphviz(treeclf, out_file=None,
                                    feature_names=feature,
#                                     class_names=['0', '1'], filled = True))
                                    class_names=['0', '1'], filled = True))
svg = SVG(graph.pipe(format='svg'))
display(svg)

graph.format = 'png'
graph.view(filename='digraph5', directory='./')

# Get a random sample to see how it does
# i = random.randint(0, n)
i = 42 # an interesting case
# i = 15 # an interesting case
sample = data.iloc[i]
print('Sample patient {} to test on. Tree classified as {}'.format(i, treeclf.predict([sample[feature]])))
display(sample)

### Split data

In [ ]:
X = data.drop(columns=['Outcome'])
y = data.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y)

### Scale data

In [ ]:
scaler = StandardScaler() # Fit only to the training data
scaler.fit(X_train) # Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Run classfier

In [ ]:
mlp0 = MLPClassifier(hidden_layer_sizes=(2),max_iter=500)
mlp1 = MLPClassifier(hidden_layer_sizes=(3, 11, 10),max_iter=500)
mlp2 = MLPClassifier(hidden_layer_sizes=(6, 9, 12, 7),max_iter=500)

### Fit data

In [ ]:
mlp0.fit(X_train,y_train)
mlp1.fit(X_train,y_train)
mlp2.fit(X_train,y_train)

### Predict on test data

In [ ]:
predictions0 = mlp0.predict(X_test)
predictions1 = mlp1.predict(X_test)
predictions2 = mlp2.predict(X_test)

### Print confusion matrix

In [ ]:
print(confusion_matrix(y_test,predictions0))
print(confusion_matrix(y_test,predictions1))
print(confusion_matrix(y_test,predictions2))

### Print classification report

In [ ]:
print(classification_report(y_test,predictions0))
print(classification_report(y_test,predictions1))
print(classification_report(y_test,predictions2))

### Print coefficients of mlp0

In [ ]:
print('This dataset has {} input nodes and {} output node(s)'.format(len(X.columns), len(y.unique())))
print('There are {} 2D arrays of coefficients, one for each layer'.format(len(mlp0.coefs_)))
print('The layers have the following number of coefficients: {}')
for l in range(len(mlp0.coefs_)):
    m = len(mlp0.coefs_[l])
    n = len(mlp0.coefs_[l][0])
    print('  {}: {}x{} ({} nodes feeding into a layer of {} nodes)'.format(l, m, n, m, n))
# Print the actual coefficients
# print(mlp.coefs_)

print()
print('There are {} 1D arrays of intercepts, one for each layer'.format(len(mlp0.intercepts_)))
print('Each layer has {} intercepts, one for each node'.format([len(mlp0.intercepts_[l]) for l,_ in enumerate(mlp0.intercepts_)]))

### Print coefficients of mlp1

In [ ]:
print('This dataset has {} input nodes and {} output node(s)'.format(len(X.columns), len(y.unique())))
print('There are {} 2D arrays of coefficients, one for each layer'.format(len(mlp1.coefs_)))
print('The layers have the following number of coefficients: {}')
for l in range(len(mlp1.coefs_)):
    m = len(mlp1.coefs_[l])
    n = len(mlp1.coefs_[l][0])
    print('  {}: {}x{} ({} nodes feeding into a layer of {} nodes)'.format(l, m, n, m, n))
# Print the actual coefficients
# print(mlp.coefs_)

print()
print('There are {} 1D arrays of intercepts, one for each layer'.format(len(mlp1.intercepts_)))
print('Each layer has {} intercepts, one for each node'.format([len(mlp1.intercepts_[l]) for l,_ in enumerate(mlp1.intercepts_)]))

### Print coefficients of mlp2

In [ ]:


print('This dataset has {} input nodes and {} output node(s)'.format(len(X.columns), len(y.unique())))
print('There are {} 2D arrays of coefficients, one for each layer'.format(len(mlp2.coefs_)))
print('The layers have the following number of coefficients: {}')
for l in range(len(mlp2.coefs_)):
    m = len(mlp2.coefs_[l])
    n = len(mlp2.coefs_[l][0])
    print('  {}: {}x{} ({} nodes feeding into a layer of {} nodes)'.format(l, m, n, m, n))
# Print the actual coefficients
# print(mlp.coefs_)

print()
print('There are {} 1D arrays of intercepts, one for each layer'.format(len(mlp2.intercepts_)))
print('Each layer has {} intercepts, one for each node'.format([len(mlp2.intercepts_[l]) for l,_ in enumerate(mlp2.intercepts_)]))



### Plot mlp0

In [ ]:


# you may need to install networkx with pip
import networkx as nx
import colorsys

mlp = mlp0

def show_ann(mlp):
    hidden_layers_n = len(mlp.coefs_)-1
    layers_n = hidden_layers_n + 2
    input_neurons_n = len(mlp.coefs_[0])
    hidden_neurons_n = [len(mlp.coefs_[i+1]) for i in range(hidden_layers_n)]
    output_neurons_n = len(mlp.coefs_[-1][0])

    G = nx.DiGraph()
    pos = {}

    # Create the neurons of the input layer
    for i in range(input_neurons_n):
        pos['Layer0_{}'.format(i)] = (i,layers_n-1)

    for j in range(hidden_layers_n):
        # Create the neurons of the j'th hidden layer
        prev_layer = j
        cur_layer = j+1
        if (j == 0):
            prev_size = input_neurons_n
        else:
            prev_size = hidden_neurons_n[j-1]
        for i in range(hidden_neurons_n[j]):
            pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
            for k in range(prev_size):
                w = mlp.coefs_[prev_layer][k][i]
                G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    # Create the neurons of the output layer
    prev_layer = hidden_layers_n
    cur_layer = hidden_layers_n+1
    for i in range(output_neurons_n):
        pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
        for k in range(hidden_neurons_n[-1]):
            w = mlp.coefs_[prev_layer][k][i]
            G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    edges = G.edges()
    colors = [colorsys.hsv_to_rgb(0 if G[u][v]['weight'] < 0 else 0.65,
                                  1,#min(1, abs(G[u][v]['weight'])),
                                  1) for u,v in edges]
    weights = [abs(G[u][v]['weight'])*2 for u,v in edges]

    nx.draw(G, pos, node_color='y', node_size=450, width=weights, edge_color=colors)
    
show_ann(mlp)



### Plot mlp1

In [ ]:
# you may need to install networkx with pip
import networkx as nx
import colorsys

mlp = mlp1

def show_ann(mlp):
    hidden_layers_n = len(mlp.coefs_)-1
    layers_n = hidden_layers_n + 2
    input_neurons_n = len(mlp.coefs_[0])
    hidden_neurons_n = [len(mlp.coefs_[i+1]) for i in range(hidden_layers_n)]
    output_neurons_n = len(mlp.coefs_[-1][0])

    G = nx.DiGraph()
    pos = {}

    # Create the neurons of the input layer
    for i in range(input_neurons_n):
        pos['Layer0_{}'.format(i)] = (i,layers_n-1)

    for j in range(hidden_layers_n):
        # Create the neurons of the j'th hidden layer
        prev_layer = j
        cur_layer = j+1
        if (j == 0):
            prev_size = input_neurons_n
        else:
            prev_size = hidden_neurons_n[j-1]
        for i in range(hidden_neurons_n[j]):
            pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
            for k in range(prev_size):
                w = mlp.coefs_[prev_layer][k][i]
                G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    # Create the neurons of the output layer
    prev_layer = hidden_layers_n
    cur_layer = hidden_layers_n+1
    for i in range(output_neurons_n):
        pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
        for k in range(hidden_neurons_n[-1]):
            w = mlp.coefs_[prev_layer][k][i]
            G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    edges = G.edges()
    colors = [colorsys.hsv_to_rgb(0 if G[u][v]['weight'] < 0 else 0.65,
                                  1,#min(1, abs(G[u][v]['weight'])),
                                  1) for u,v in edges]
    weights = [abs(G[u][v]['weight'])*2 for u,v in edges]

    nx.draw(G, pos, node_color='y', node_size=450, width=weights, edge_color=colors)
    
show_ann(mlp)

In [ ]:
# you may need to install networkx with pip
import networkx as nx
import colorsys

mlp = mlp2

def show_ann(mlp):
    hidden_layers_n = len(mlp.coefs_)-1
    layers_n = hidden_layers_n + 2
    input_neurons_n = len(mlp.coefs_[0])
    hidden_neurons_n = [len(mlp.coefs_[i+1]) for i in range(hidden_layers_n)]
    output_neurons_n = len(mlp.coefs_[-1][0])

    G = nx.DiGraph()
    pos = {}

    # Create the neurons of the input layer
    for i in range(input_neurons_n):
        pos['Layer0_{}'.format(i)] = (i,layers_n-1)

    for j in range(hidden_layers_n):
        # Create the neurons of the j'th hidden layer
        prev_layer = j
        cur_layer = j+1
        if (j == 0):
            prev_size = input_neurons_n
        else:
            prev_size = hidden_neurons_n[j-1]
        for i in range(hidden_neurons_n[j]):
            pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
            for k in range(prev_size):
                w = mlp.coefs_[prev_layer][k][i]
                G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    # Create the neurons of the output layer
    prev_layer = hidden_layers_n
    cur_layer = hidden_layers_n+1
    for i in range(output_neurons_n):
        pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
        for k in range(hidden_neurons_n[-1]):
            w = mlp.coefs_[prev_layer][k][i]
            G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    edges = G.edges()
    colors = [colorsys.hsv_to_rgb(0 if G[u][v]['weight'] < 0 else 0.65,
                                  1,#min(1, abs(G[u][v]['weight'])),
                                  1) for u,v in edges]
    weights = [abs(G[u][v]['weight'])*2 for u,v in edges]

    nx.draw(G, pos, node_color='y', node_size=450, width=weights, edge_color=colors)
    
show_ann(mlp)